# Introduktion till Apache Spark
Apache Spark är ett distribuerat ramverk för att hantera stora datamängder som är extremt hett just nu. Till skillnad mot MapReduce så använder sig Spark av minnesstrukturer vilket snabbar upp processningen avsevärt, särskilt på iterativa flöden som machine learning. 

Spark innehåller komponenter för allt från strömprocessning, sql och dataframes, machine learning och grafhantering. Vi kommer inte hinna gå igenom allt men ska försöka täcka in de områden som jag bedömer är de vi kommer att arbeta mest med vilket är dataframes och SQL.

Läs gärna mer på Sparks webbsida <a href="http://spark.apache.org/">http://spark.apache.org/</a>

## Läsa och skriva data
I Jupyter har vi tillgång till i princip all data vi har på Svenska Spel. Vi kan använda oss av både Spark och Pandas för att läsa och skriva data. Med tanke på de generella datavolymer vi kommer att behöva hantera kommer Spark att vara det ramverk som används oftast.

### Läsa data från Hive/Hadoop
Spark har inbyggt stöd för att läsa från och skriva till Hive. För att arbeta med data lagrad i Hive behöver man starta upp ett `HiveContext()` enligt nedan.

In [ ]:
hc = HiveContext(sc)

För att lista tillgängliga tabeller kan man använda funktionen `.tableNames()` vilket returnerar en lista på tabeller i ett givet schema.

In [ ]:
hc.tableNames('analytics_prod_11')

För att arbeta med data från en tabell kan man antingen skapa en data frame direkt från en tabell eller så kan vi använda Spark för att ställa frågor via SQL. För att skapa en data frame direkt används funktionen `.table()` genom att ange `schema.tabell` så här.

In [ ]:
oms = hc.table('analytics_prod_11.oms')

In [ ]:
oms.show(5)

För att ställa en sqlfråga mot Hive använder vi funktionen `.sql()` som tar en fråga som input och levererar resultatet i form av en data frame. Ofta är det enklast att spara frågan i en separat variabel som sedan skickas in i funktionen enligt nedan.

In [ ]:
query = """

select gem_product_id, sum(wager_sg_1_sek) as sg1, sum(wager_sg_2_sek) as sg2
from analytics_prod_11.bet
where dt >= '2016-01-01'
group by gem_product_id

"""

In [ ]:
result = hc.sql(query)

In [ ]:
result.show(5)

### Skriva data till Hive
Om vi vill skriva exempelvis resultat ovan till en tabell i Hive kan vi göra det via funktionen `.write.saveAsTable()` enligt nedan. Funktionen tar argumenten `schema.tabell`, `format` och `mode`.

In [ ]:
result.write.saveAsTable('user_dabc.results', 'orc', 'overwrite')

Vi kan validera att vi lyckats genom att se att tabellen finns i schemat.

In [ ]:
hc.tableNames('user_dabc')

### Läsa filer från Oracle eller annan jdbc-källa
Spark kan läsa data via jdbc genom att ange den tabell man vill komma åt enligt nedan.

In [ ]:
d_customer = hc.read.jdbc(url='jdbc:oracle:thin:dabc/Sommar2014@hexa-scan.vby.svenskaspel.se:1521/dwdb', 
                          table='dmuser.d_customer', 
                          properties={'driver':'oracle.jdbc.driver.OracleDriver'}
                         )

In [ ]:
d_customer.printSchema()

In [ ]:
d_customer.select('CUSTOMER_KEY', 'GENDER_NAME', 'EMPLOYEE').show(5)

Det går också att istället för en tabell ange en fråga i form av en subquery, med andra ord en selectsats med paranteser.

In [ ]:
subquery = """

(select year_no, f.date_key as transaction_date, customer_number as customer_id, sum(amount_rake) as rake, 'poker' as product_id 
from dmuser.v_cube_d_poker_sales f
inner join dmuser.d_date d on f.date_key = d.date_key
inner join dmuser.d_customer c on f.customer_key = c.customer_key
where year_no >= 2014
group by year_no, f.date_key, customer_number)

"""

In [ ]:
poker = hc.read.jdbc(url='jdbc:oracle:thin:dabc/Sommar2014@hexa-scan.vby.svenskaspel.se:1521/dwdb', 
                     table=subquery, 
                     properties={'driver':'oracle.jdbc.driver.OracleDriver'}
                    )

In [ ]:
poker.printSchema()

In [ ]:
poker.show(5)

Om man ska köra många frågor är det ju lättare att lägga `url` och `properties` i variabler som går att återanvända.

In [ ]:
url = 'jdbc:oracle:thin:dabc/Sommar2014@hexa-scan.vby.svenskaspel.se:1521/dwdb'
properties = {'driver':'oracle.jdbc.driver.OracleDriver'}

In [ ]:
tmp = hc.read.jdbc(url=url, 
                     table='dmuser.d_product', 
                     properties=properties
                    )

tmp.printSchema()

In [ ]:
tmp.write.saveAsTable('user_dabc.tmp_product', 'orc', 'overwrite')

### Skriva till Oracle
Spark har från och med version 1.6.2 möjlighet att skriva data till Oracle efter att tidigare ha haft problem med datatyper. 

In [ ]:
poker.write.jdbc(url, 'sasuser.dabc_jdbc_test', 'overwrite', properties)

### Skriva resultatet av hivefrågor till fil (fungerar enbart via ssh till hdg01 just nu)

In [ ]:
!hive -e 'select * from analytics_prod_11.oms' > /home/dabc/temp.tsv

### Läsa filer i vår data lake
På Svenska Spel har vi en data lake-strategi vilket innebär att vi lagrar data i dess ursprungsform vilket i många fall är jsonstrukturer. En av de stora fördelarna med Spark data frames är att det finns mycket bra stöd för att tolka och bearbeta data som inte har en tabulär struktur.

Då hadoop lagrar vårt data i formatet `sequencefile` kan vi göra enligt nedan för att hitta och läsa datat. Vi kan börja med att använda hdfs-klienten i Jupyter för att lista hadoops filstruktur:

In [ ]:
!hdfs dfs -ls /

Vår data lake ligger under katalogen svsdata.

In [ ]:
!hdfs dfs -ls /svsdata

Här ser vi flera foldrar och den vi är intresserad av är argon_prod. Genom att ange `| head` kan vi begränsa antalet poster som listas.

In [ ]:
!hdfs dfs -ls /svsdata/argon_prod | head

Om vi tittar under en speciell katalog ser vi att datat ligger partitionerat per datum.

In [ ]:
!hdfs dfs -ls /svsdata/argon_prod/ItsRegWager | head

För att läsa dessa tabeller kan vi sätta upp en läsare mot våra sekvensfiler så här. Notera de olika sätt vi kan strypa hur mycket data vi läser upp med hjälp av olika wildcards.

In [ ]:
# För att läsa ett helt år
# sc.sequenceFile('/svsdata/argon_prod/ItsRegWager/dt=2006*')

# För att läsa ett flera år
# sc.sequenceFile('/svsdata/argon_prod/ItsRegWager/dt={2015,2016}*')

# För att läsa en månad
# sc.sequenceFile('/svsdata/argon_prod/ItsRegWager/dt=2016-04*')

# Eller för att läsa ett specifik datum
seq = sc.sequenceFile('/svsdata/argon_prod/ItsRegWager/dt=2006-03-20')

I det här läget är datat lagrat som ett dataset av typen RDD (resilient distributed dataset) vilket är Sparks primära datastruktur. 

In [ ]:
type(seq)

Vi kan titta på första raden för att se hur det ser ut med funktionen `.first()`. Som vi ser är varje rad en key-value-struktur där datat är representerat som json.

In [ ]:
seq.first()

Då vi bara är intresserade av jsonstrukturen kan vi enkelt komma åt den genom `.values()`.

In [ ]:
seq_v = seq.values()
seq_v.first()

In [ ]:
seq_v.count()

Nu när vi har vårt data i en RDD som består av json så kan vi enkelt skapa upp en data frame med funktionen `.jsonRDD()`. Den här funktionen kommer att scanna igenom datat och derivera fram ett schema som vi sedan kan använda oss av för att bearbeta datat. 

Notera att det är viktigt att ange parametern `samplingRatio` för att berätta hur stor del av datat Spark ska använda för att tolka datat. I och med att vi har variabla strukturer behöver Spark i en del fall läsa hela datasetet medans det i andra fall räcker med en liten andel. Här får man ibland prova sig fram.

I och med att vi i det här fallet har 154052 rader i datat kan vi nöja oss med en mindre andel. 

In [ ]:
bets = hc.jsonRDD(seq_v, samplingRatio=0.1)

När Spark har tolkat datat färdigt kan vi se vad vi får tillbaka för schema med funktionen `.printSchema()`.

In [ ]:
bets.printSchema()

Det går som vanligt att komprimera läsning och tolkning till en instruktion, exempelvis så här.

In [ ]:
bet = hc.jsonRDD(sc.sequenceFile('/svsdata/argon_prod/ItsRegWager/dt=2016-03*').values(), samplingRatio=0.1)

Vi kan visa de översta 5 raderna som vanligt.

In [ ]:
bet.show(5)

## Spark data frames
Spark har liksom Pandas ett koncept för data frames. Spark har försökt att i möjligaste mån ligga så nära Pandas som möjligt vilket gör att det är relativt enkelt att komma igång med Spark om man tidigare har arbetat med Pandas. Vi börjar med att läsa in lite data som vi kan arbeta med.

In [ ]:
bet = hc.table('analytics_prod_11.bet')

In [ ]:
type(bet)

In [ ]:
bet.printSchema()

Vi kan även returnera alla kolumner i form av en lista via attributet `.columns`. Detta kan vara användbart om vi vill arbeta med metadata programmatiskt för att exempelvis loopa och applicera funktioner på många kolumner.

In [ ]:
cols = bet.columns
cols

In [ ]:
for col in cols:
    print 'Do something with ' + col

Vi kan enkelt räkna antalet rader med funktionen `.count()`.

In [ ]:
bet.count()

För att droppa en rad använder vi funktionen `.drop()`. Funktionen returnerar en ny data frame så vi måste deklarera en ny variabel för att ta emot detta. 

In [ ]:
dropped = bet.drop('addon')
dropped.printSchema()

Spark använder sig av något som kallas för `lazy evaluation` vilket innebär att inga operationer exekveras förrän man begär en outputoperation. Detta skiljer sig från pandas där man har resultatset i minnet.

### Selekteringar i Spark
Om vi exempelvis vill välja ut några kolumner i Spark kan vi köra funktionen `.select()` och skicka in de kolumnnamn vi vill ha. Som ni märker sker ingen exekvering i det här läget utan Spark bygger endast upp en del i ett exekveringsträd.

In [ ]:
selection = bet.select('wager_serial', 'customer_id', 'wager_sg_1_sek')

Vi kan se att Spark ändå har gjort en transformation genom `.printSchema()`.

In [ ]:
selection.printSchema()

Om vi nu vill se resultatet så kommer Spark att köra allt ovan och returnera ett resultatset enligt begärt.

In [ ]:
selection.limit(10).show()

### Filtreringar och reducering av datamängder
För att filtrera på data kan vi använda funktionen `.filter()` och lägga in det villkor vi vill köra enligt nedan.

In [ ]:
filtered = bet.filter("dt >= '2016-01-01'")
filtered.show(5)

Vi kan också applicera funktionerna `.limit()` eller `.sample()` för att reducera mängden data. 

In [ ]:
filtered.limit(100).show(5)

In [ ]:
filtered.sample(False, 0.01).show(5)

Om vi har en mindre mängd data så kan vi enkelt konvertera en data frame i Spark till en data frame i Pandas. Det är dock viktigt att tänka på vilka volymer data man arbetar med i det här fallet. Om vi exempelvis skulle försöka lyfta över hela tabellen bet som är på 3,5 miljarder rader kommer vi att köra slut på minne och krascha Sparkjobbet. 

En vettig volym ligger på 2-3 miljoner rader. Fördelen med att lyfta till Pandas är att vi får tillgång till ett rikare utbud av funktioner och att Pandas visar data på ett sätt som passar bättre i en notebook.

om vi som ovan tar ett sample på 1% av datavolymen ser vi att vi landar på en hanterbar volym data. 

In [ ]:
sample = filtered.sample(False, 0.01)
sample.count()

För att lyfta detta till Pandas använder vi funktionen `.toPandas()`. Det tar en liten stund att flytta datat från klustret till jupyterservern men när det väl är flyttat går det mycket snabbare att arbeta med. 

In [ ]:
pandas_df = sample.toPandas()

In [ ]:
pandas_df.head()

Om vi tittar på datatyperna ser vi att vi har en data frame på Spark och en på Pandas.

In [ ]:
print type(sample)
print type(pandas_df)

Vi kan använda oss av funktionen `.info()` på pandas_df för att se hur mycket minne den tar. I det här fallet 123mb vilket inte är särskilt mycket.

In [ ]:
pandas_df.info()

### Sorting
Sortering av en data frame kan göras med funktionen `.sort()` vilken tar valfritt antal kolumner som input. Kolumner i Spark deklareras på samma sätt som Pandas enligt `df['kolumn']`.

In [ ]:
bet = bet.filter("dt > '2015-12-31'")

In [ ]:
bet['dt']


För att ange sortering på kolumnen kan `.asc()` eller `.desc()` användas.

In [ ]:
bet['dt'].desc()

För att sortera tabellen `bet` fallande efter datum skriver man såhär.

In [ ]:
bet.sort(bet['dt'].desc()).show(5)

Vi kan sortera på fler begrepp så här.

In [ ]:
bet.sort(bet['dt'].desc(), bet['customer_id'].asc()).show()

### SQL med Spark
Ovan har vi använt oss av Sparks objektorienterade API mot våra data frames. Vi kan också enkelt registrera en data frame som en temporär tabell som vi kan skriva SQL mot. För detta anävnder vi oss av funktionen `.registerTempTable()` enligt nedan.

In [ ]:
sample.registerTempTable('tmp')

När vi har gjort detta kan vi ställa SQL-frågor mot tabellen som om den vore en fysisk tabell.

In [ ]:
hc.sql("""

select dt, sum(wager_sg_1_sek) as sales from tmp
group by dt
order by dt asc
limit 10

""").toPandas()